In [ ]:
Q14. What is the function of PySpark's pivot() method? 
The pivot() method in PySpark is used to rotate/transpose data 
from one column into many Dataframe columns and back using 
the unpivot() function (). Pivot() is an aggregation in which the 
values of one of the grouping columns are transposed into 
separate columns containing different data. 
To get started, let's make a PySpark DataFrame. 
import pyspark 
from pyspark.sql import SparkSession 
from pyspark.sql.functions import expr 
#Create spark session 
data = [("Banana",1000,"USA"), ("Carrots",1500,"USA"), 
("Beans",1600,"USA"), \ 
      ("Orange",2000,"USA"),("Orange",2000,"USA"),("Banana",4
00,"China"), \ 
      ("Carrots",1200,"China"),("Beans",1500,"China"),("Orange",
4000,"China"), \ 
      ("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Bea
ns",2000,"Mexico")] 
columns= ["Product","Amount","Country"] 
df = spark.createDataFrame(data = data, schema = columns) 
df.printSchema() 
df.show(truncate=False) 
Output- 
Created by Rahul Pupreja, linkedin.com/in/rahulpupreja 
 
 
To determine the entire amount of each product's exports to 
each nation, we'll group by Product, pivot by Country, and sum 
by Amount. 
pivotDF = 
df.groupBy("Product").pivot("Country").sum("Amount") 
pivotDF.printSchema() 
pivotDF.show(truncate=False) 
This will convert the nations from DataFrame rows to columns, 
resulting in the output seen below. Wherever data is missing, it 
is assumed to be null by default. 


In [ ]:
Q15. In PySpark, how do you generate broadcast 
variables? Give an example. 
Broadcast variables in PySpark are read-only shared variables 
that are stored and accessible on all nodes in a cluster so that 
processes may access or use them. Instead of sending this 
information with each job, PySpark uses efficient broadcast 
algorithms to distribute broadcast variables among workers, 
lowering communication costs. 
The broadcast(v) function of the SparkContext class is used to 
generate a PySpark Broadcast. This method accepts the 
broadcast parameter v. 
Generating broadcast in PySpark Shell: 
broadcastVariable = sc.broadcast(Array(0, 1, 2, 3)) 
broadcastVariable.value 
PySpark RDD Broadcast variable example 
spark=SparkSession.builder.appName('SparkByExample.com')
.getOrCreate() 
states = {"NY":"New York", "CA":"California", "FL":"Florida"} 
Created by Rahul Pupreja, linkedin.com/in/rahulpupreja 
 
broadcastStates = spark.sparkContext.broadcast(states) 
data = [("James","Smith","USA","CA"), 
    ("Michael","Rose","USA","NY"), 
    ("Robert","Williams","USA","CA"), 
    ("Maria","Jones","USA","FL") 
  ] 
rdd = spark.sparkContext.parallelize(data) 
def state_convert(code): 
    return broadcastState.value[code] 
res = rdd.map(lambda a: 
(a[0],a[1],a[2],state_convert(a{3]))).collect() 
print(res) 
PySpark DataFrame Broadcast variable example 
spark=SparkSession.builder.appName('PySpark broadcast 
variable').getOrCreate() 
states = {"NY":"New York", "CA":"California", "FL":"Florida"} 
broadcastStates = spark.sparkContext.broadcast(states) 
data = [("James","Smith","USA","CA"), 
    ("Michael","Rose","USA","NY"), 
    ("Robert","William","USA","CA"), 
    ("Maria","Jones","USA","FL") 
  ] 
Created by Rahul Pupreja, linkedin.com/in/rahulpupreja 
 
columns = ["firstname","lastname","country","state"] 
df = spark.createDataFrame(data = data, schema = columns) 
df.printSchema() 
df.show(truncate=False) 
def state_convert(code): 
    return broadcastState.value[code] 
res = df.rdd.map(lambda a: 
(a[0],a[1],a[2],state_convert(a[3]))).toDF(column) 
res.show(truncate=False

In [ ]:
Q1. You have a cluster of ten nodes with each node having 
24 CPU cores. The following code works, but it may crash 
on huge data sets, or at the very least, it may not take 
advantage of the cluster's full processing capabilities. 
Which aspect is the most difficult to alter, and how would 
you go about doing so? 
def cal(sparkSession: SparkSession): Unit = { val 
NumNode = 10 val userActivityRdd: RDD[UserActivity] = 
readUserActivityData(sparkSession) . 
repartition(NumNode) val result = userActivityRdd .map(e 
=> (e.userId, 1L)) . reduceByKey(_ + _) result .take(1000) } 
The repartition command creates ten partitions regardless of 
how many of them were loaded. On large datasets, they might 
get fairly huge, and they'll almost certainly outgrow the RAM 
allotted to a single executor. 
Created by Rahul Pupreja, linkedin.com/in/rahulpupreja 
 
In addition, each executor can only have one partition. This 
means that just ten of the 240 executors are engaged (10 
nodes with 24 cores, each running one executor). 
If the number is set exceptionally high, the scheduler's cost in 
handling the partition grows, lowering performance. It may even 
exceed the execution time in some circumstances, especially 
for extremely tiny partitions. 
The optimal number of partitions is between two and three 
times the number of executors. In the given scenario, 600 = 10 
× 24 x 2.5 divisions would be appropriate.

In [ ]:
Q2. Explain the following code and what output it will yield- 
case class User(uId: Long, uName: String) case class 
UserActivity(uId: Long, activityTypeId: Int, 
timestampEpochSec: Long) val LoginActivityTypeId = 0 val 
LogoutActivityTypeId = 1 private def 
readUserData(sparkSession: SparkSession): RDD[User] = { 
sparkSession.sparkContext.parallelize( Array( User(1, 
"Doe, John"), User(2, "Doe, Jane"), User(3, "X, Mr.")) ) } 
private def readUserActivityData(sparkSession: 
SparkSession): RDD[UserActivity] = { 
sparkSession.sparkContext.parallelize( Array( 
UserActivity(1, LoginActivityTypeId, 1514764800L), 
UserActivity(2, LoginActivityTypeId, 1514808000L), 
UserActivity(1, LogoutActivityTypeId, 1514829600L), 
UserActivity(1, LoginActivityTypeId, 1514894400L)) ) } def 
calculate(sparkSession: SparkSession): Unit = { val 
userRdd: RDD[(Long, User)] = 
readUserData(sparkSession).map(e => (e.userId, e)) val 
userActivityRdd: RDD[(Long, UserActivity)] = 
readUserActivityData(sparkSession).map(e => (e.userId, e)) 
val result = userRdd .leftOuterJoin(userActivityRdd) 
.filter(e => e._2._2.isDefined && e._2._2.get.activityTypeId 
== LoginActivityTypeId) .map(e => (e._2._1.uName, 
e._2._2.get.timestampEpochSec)) .reduceByKey((a, b) => if 
(a < b) a else b) result .foreach(e => println(s"${e._1}: 
${e._2}")) } 
The primary function, calculate, reads two pieces of data. 
(They are given in this case from a constant inline data 
Created by Rahul Pupreja, linkedin.com/in/rahulpupreja 
 
structure that is transformed to a distributed dataset using 
parallelize.) Each of them is transformed into a tuple by the 
map, which consists of a userId and the item itself. To combine 
the two datasets, the userId is utilised. 
All users' login actions are filtered out of the combined dataset. 
The uName and the event timestamp are then combined to 
make a tuple. 
This is eventually reduced down to merely the initial login 
record per user, which is then sent to the console. 
The following will be the yielded output- 
Doe, John: 1514764800 
Doe, Jane: 1514808000